# Učitavanje biblioteka za rad 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

random.seed(1)

In [2]:
# broj indeksa za studenta Jovan Dmitrović
ind = '2021/3096'

# odredjivanje algoritama koje je potrebno implementirati
B3 = int(ind[-2])
B4 = int(ind[-1])

even_num = lambda num: True if num%2 == 0 else False

algos_dict = {(False, False): ['Softmax logisticka regresija', 'Gauss-ovski naivni Bayes'],
             (False, True) : ['Softmax logisticka regresija', 'Gauss-ovska diskriminativna analiza'],
             (True, False) : ['Logisticka regresija', 'Gauss-ovski naivni Bayes'],
             (True, True)  : ['Logisticka regresija', 'Gauss-ovska diskriminativna analiza']}

algos_selected = algos_dict[(even_num(B3), even_num(B4))]

print(f'Na osnovu broja indeksa, potrebno je implementirati sledeca 2 algoritma: \n - {algos_selected[0]} i \n - {algos_selected[1]}')

Na osnovu broja indeksa, potrebno je implementirati sledeca 2 algoritma: 
 - Softmax logisticka regresija i 
 - Gauss-ovska diskriminativna analiza


# Učitavanje podataka 

In [3]:
def get_classes_info(y: np.ndarray, show_text: bool=True) -> int:
    
    """
        Funkcija za izvlacenje klasa i prebrojavanja primera
        sa istom klasom
        
        params: 
            - y: vektor klasa
                               
        :return:
            - k: broj klasa
    """
    # dohvatanje svih vrednosti klasa
    classes = []
    for _class in y:
        if _class not in classes:
            classes.append(int(_class))
            
    # prebrojavanje predstavnika klasa
    classes_dict = dict.fromkeys(classes)
    for _class in classes:
        class_count = np.count_nonzero(y == _class)
        classes_dict[_class] = class_count
        
    # broj klasa
    k = len(classes_dict.keys())
        
    # ispis podataka o klasama u dataset-u
    if show_text:
        print('-------------------------------------------------------')
        print(f'Ukupan broj klasa kojima pripadaju ucitani primeri je: {k}\n')
        print('Ucitane klase i brojevi njhovih predstavnika/primera su: ')
        for _class in classes_dict.keys():
            print(f'\tKlasa {_class}: {classes_dict[_class]} primera')
        print('-------------------------------------------------------')
    
    return k

In [4]:
data = pd.read_csv('multiclass_data.csv').to_numpy()
X, y = data[:,0:5], data[:,5]

print('-------------------Ucitani su podaci-------------------')
print(f'Ukupan broj ucitanih primera je: {X.shape[0]}')
print(f'Ukupan broj ucitanih prediktora (dim(X)) je: {X.shape[1]}')

k = get_classes_info(y)

-------------------Ucitani su podaci-------------------
Ukupan broj ucitanih primera je: 177
Ukupan broj ucitanih prediktora (dim(X)) je: 5
-------------------------------------------------------
Ukupan broj klasa kojima pripadaju ucitani primeri je: 3

Ucitane klase i brojevi njhovih predstavnika/primera su: 
	Klasa 0: 58 primera
	Klasa 1: 71 primera
	Klasa 2: 48 primera
-------------------------------------------------------


Klase nisu "savršeno balansirane" (pojednaka brojčane), medjutim ne može se reći ni da su nebalansirane, s obzirom na to da broj primera svake pojedinačne klase nije drastično mali, npr. klasa 2 ima 3 odbirka. \
Ovaj problem nebalansiranih klasa moguće "rešiti", tj. unekoliko poboljšati vršenjem takozvanog *oversampling*-a, ne bi li se broj primera najnemnogobrojnije klase iole mogao izjednačiti sa ostalim klasama, ili otežinjavanjem primera najnemnogobrojnije klase prilikom računanje funkcije gubitka. \
*Oversampling* podrazumeva da se pomoću estimirane raspodele podataka najnemnogobrojnije klase, generišu neki novi podaci koji navodno pripadaju toj klasi (često se za ovaj postupak upotrebljava k-Nearest-Neighbours algoritam). 

# Teorijski osvrt

Logistička regresija je algoritam za binarnu klasifikaciju. Ne može se primeniti direktno u slučajevima kada postoji više od dve ciljne klase. Jedna mogućnost je tzv "jedan-protiv-ostalih" strategija: za svaku klasu projektuje se po jedan binarni klasifikator, koji odlučuje da li primer pripada toj klasi ili ne. Druga mogućnost je "jedan-protiv-jednog", gde projektujemo po jedan klasifikator za svaki par klasa.


**Multinomijalna logistička regresija**, poznatija pod nazivom **Softmax** klasifikator, prirodno podržava probleme sa više od 2 ciljne klase. U pitanju je generalizovani linearni model koji izlaznoj promenljivoj $y$ pridružuje multinomijalnu raspodelu:
$$
\Pr(Y=i) = \phi_i, \quad i=1, \ldots, k
\\
\sum_{i=1}^k \phi_i = 1
$$

# Implementacija potrebnih funkcija

In [5]:
def read_data(path: str='multiclass_data.csv', separator_index: int=5) -> tuple:
    
    """
        Funkcija za ucitavanje podataka na osnovu kojih se
        kreira model.
        
        :params: 
            - path: putanja do .csv fajla sa podacima
            - separator_index: broj kolone do koje se nalaze
                               ulazni podaci, odnosno od koje
                               pocinju izlazni podaci
                               
        :return:
            - X: matrica primera svih prediktora
            - y: vektor izlaza
    """
    
    data = pd.read_csv(path).to_numpy()
    
    # razdvajanje ulaznih i izlaznih promenljivih
    X, y = data[:,0:separator_index], data[:,separator_index]
    
    return X, y

In [6]:
def shuffle_data(X: np.ndarray, y: np.ndarray) -> tuple:
    
    """
        Funkcija za mesanje primera. Klase su u uzlanom .csv fajlu skoncentrisane
        u grupe prilikom ucitavanja. Razdvajanje podataka na trening i na test
        podatke dovelo bi do nebalansiranja klasa, sto se izbegava mesanjem klasa.
        
        :params:
            - X: matrica primera svih prediktora
            - y: vektor klasa
                           
        :return:
            - X: izmesana matrica primera svih prediktora
            - y: izmesan vektor klasa
    """

    random.seed(1)
    
    # dohvatanje broja primera u ulaznim podacima
    m = X.shape[0]
    
    # kreiranje niza indeksa
    indeces = np.arange(0,m)

    # mesanje indeksa
    shuffled_indeces = random.sample(list(indeces), m)

    return X[shuffled_indeces], y[shuffled_indeces]

In [7]:
def train_test_data_split(X: np.ndarray, y: np.ndarray, split_ratio: float=0.8) -> tuple:
    
    """
        Funkcija za mesanje klasa i splitovanje svih dostupnih podataka na 
        trening/obucavajuce i test podatke prema zadatom postotku (split_ratio).
        
        :params:
            - X: matrica primera svih prediktora
            - y: vektor izlaza
            - split_ratio: vrednost iz intervala [0,1] koja oznacava procenat
                           trening podataka
                           
        :return:
            - X_train: matrica primera svih prediktora za trening skup
            - y_train: vektor izlaza za trening skup
            - X_test: matrica primera svih prediktora za test skup
            - y_test: vektro izlaza za test skup
    """
    
    random.seed(1)
    
    m = X.shape[0]
    
    train_set_size = int(m*split_ratio)
    
    ind = np.arange(0,m).tolist()
    
    # odredjivanje indeksa za trening skup
    train_set_ind = random.sample(ind, k=train_set_size)
    
    # odredjivanje indeksa za test skup
    test_set_ind = list(set(ind) - set(train_set_ind))
    
    # mesanje podataka
    X, y = shuffle_data(X, y)
    
    # podela podataka na trening i test podatke
    X_train, y_train = X[train_set_ind], y[train_set_ind]
    X_test, y_test = X[test_set_ind], y[test_set_ind]
    
    return X_train, y_train, X_test, y_test

In [8]:
def get_statistics(X: np.ndarray) -> tuple:
    
    """
        Funkcija za dohvatanje statistika - srednja vrednost i standardna devijacija. 
        Za svaka od kolona matrice X vrsi se izracunavanje statistika, s obzirom na to da
        one predstavljaju posebne prediktore.
        
        :params:
            - X: matrica primera svih prediktora
            
        :return:
            - mu: vektor srednjih vrednosti za svaki prediktor
            - std: vektor standardnih devijacija za svaki prediktor
    """
    
    # broj prediktora
    n = X.shape[1]
    
    # vektor srednjih vrednosti za svaki prediktor ponaosob
    mu = np.zeros(n)
    
    # vektor standardnih devijacija za svaki prediktor ponaosob
    std = np.zeros(n)
    
    for i in range(n):
        mu[i] = np.mean(X[:,i])
        std[i] = np.std(X[:,i])
    
    return mu, std

In [9]:
def standardize(X: np.ndarray, mu: np.ndarray=None, std: np.ndarray=None) -> np.ndarray:
    
    """
        Funkcija za standardizaciju prediktora - postavljanje prediktora na istu skalu.
        Svaka kolona matrice X se standardizuje sa sebi svojstvenim statistikama - mu i std,
        s obzirom na to da su to posebni prediktori i da u opstem slucaju ne moraju biti na istoj skali.
        
        :params:
            - X: matrica primera svih prediktora
            - mu: vektor srednjih vrednosti za svaki prediktor ili None
            - std: vektor standardnih devijacija za svaki prediktor ili None
            
        :return:
            - X: standardizovana matrica primera svih prediktora
    """
    
    # racunanje srednjih vrednosti i st. devijacije svih prediktora
    if mu is None and std is None:
        mu, std = np.mean(X, axis=0), np.std(X, axis=0)

    # standardizacija na normalnu raspodelu
    X = (X - mu) / std # ~ N(0,1)
    
    return X

In [10]:
# ucitavanje podataka
X, y = read_data()

# broj primera
m = X.shape[0]

# broj prediktora
n = X.shape[1]

# broj klasa
k = get_classes_info(y, show_text=False)

# podela podataka na trening i test skup
X_train, y_train, X_test, y_test = train_test_data_split(X, y, split_ratio=0.8)

# podela trening podataka na trening i validacioni skup
X_train, y_train, X_val, y_val = train_test_data_split(X_train, y_train, split_ratio=0.8)

# dohvatanje sr. vrednosti i st. devijacije trening skupa
mu, std = get_statistics(X_train)

# standardizacija trening skupa 
X_train = standardize(X_train)

# standardizacija validacionog skupa sa statistikama iz trening skupa
X_val = standardize(X_val, mu, std)

In [11]:
# [TODO] - racunanje loss-a
# [TODO] - racunanje gradienta
# [TODO] - jednokoracni gradijent sa razlicitim learning_rate-om
# [TODO] - treniranje
# [TODO] - prikaz rezultata treniranja
# [TODO] - predikcija (u vidu verovatnoca za i-ti primer po svim klasama)
# [TODO] - predikcija (u vidu izvlacenja one klase sa najvecom verovatnocom)
# [TODO] - racunanje predikcija i metrika tacnosti (confusion_matrix) za trening, validacioni i test skup

Ovaj model, generalno ima 2 hiper parametra:
- red polinoma - p
- jačina regularizacije - $\lambda$.

Red polinoma je u zadatku fiksiran, tako da je potrebno samo pronaći optimlano $\lambda$.\
Optimalan parametar $\lambda$ odredjuje se unakrsnom validacijom, pri čemu vršimo pretragu po svim lambdama iz nekog opsega.

Unaksne validacija (*eng. cross validation*) jeste jedna od najboljih metoda za procenu performansi modela.\
To je postupak u kojem celi skup podataka (uglavnom skup podataka koji su rezervisani za obučavanje) podelimo na *k* delova, tzv. foldovi, jedan fold podataka proglasimo 
da je validacioni skup, a preostalih *k-1* proglasimo da pripadaju obučavajućem skupu. Zatim "istreniramo" naš model na novonastalom obučavajućem skupu (i uzmemo vrednosti metrika) i 
izracunamo vrednost adekvatne metrike tako dobijenog modela za predikcije iz validacionog skupa. To sve ponavljamo *k* puta, uzimajući uvek različiti set podataka za validacioni skup.\
Na kraju samo usrednjimo dobijene metrike i dobijamo procenu metrike (npr. srednje kvadratna greška izlaza), odnosno procenu tačnosti našeg modela na validacionom skupu. 

Ovakav postupan možemo da ponavljamo za svako $\lambda$ iz nekog opsega vrednosti. Ono $\lambda$ koje na validacionom skupu daje minimalnu (minimalnu ukoliko je metrika nenegativna kao npr. (R)MSE) grešku, proglašavamo našim optimalnim parametrom $\lambda$. 

Taj postupak vrši se priloženim kodom ispod. 

Potrebno je još spomenuti, da je, radi procene stvarne greške modela u produkciji, celokupan skup podataka potrebno je podeliti na dva podskupa:
- obučavajući - uglavnom čini od 70 do 90% ukupnih podataka
- testirajući - uglavnom čini od 10 do 30% ukupnih podataka

Na obučavajućem ili trening skupu sve vrši treniranje modela, unakrsna validacija i pronalaženje optimalnog $\lambda$, a na testirajućem skupu se vrši merenje performansi modela kroz adekvatnu metriku.

In [12]:
# ucitavanje podataka
X, y = read_data()

# podela podataka na trening i test skup
X_train, y_train, X_test, y_test = train_test_data_split(X, y, split_ratio=0.8)

# kreiranje polinomijalnih prediktora za trening skup
X_train = make_polynomial_features(X_train)

# formiranje niza svih mogucih vrednosti za hiper-parametar lambda
lambdas = np.linspace(0.1, 100, 200)

# pretraga unakrsnom validacijom za optimalno lambda
lambda_opt = optimal_lambda_search(X_train, y_train, lambdas, k=5, rmse=False, verbose=True)

NameError: name 'make_polynomial_features' is not defined

Dakle, pronadjena optimalna vrednost hiper-parametra $\lambda$ iznosi otprilike 24. Ova vrednost na prestavlja ni preveliku, ni premalu vrednost za ovaj hiper-parametar, u smislu da daje adekvatna regularizaciju modela. Premalo $\lambda$, npr. 1e-5 značilo bi da regularizacije modela uopšte nema, dok bi veliko $\lambda$, npr. 1000 značilo da se model previše regularizuje i da se preveliki znacaj pridaje parametrima modela, a ne i performansama modela.\
Zbog toga optimalna vrednost $\lambda$ hiper-parametra dovoljno dobro vrši regularizaciju, penalizujući parametre modela, ali i predikcije takvog modela su dovoljno dobre, što je suština ove pretrage. 

Potrebno je samo argumentovati da je na prikazanom grafiku sa razlogom prikazana zavisnost MSE-a od hiper-parametra $\lambda$, a ne funkcija gubitka modela, jer se na taj način odredjivanjem $\lambda$ kao argumenta koji daje minimalno MSE na validacionom skupu, nešto veći značaj pridaje tačnoj predikciji nego parametrima modela.

In [ ]:
# racunanje finalnih parametara modela
theta, theta0 = train_with_optimal_lambda(X_train, y_train, lambda_opt)

# cuvanje finalnih parametara modela
save_params(theta, theta0)

In [ ]:
# ucitavanje parametara
theta, theta0 = load_params()

# formiranje recnika za pd.DataFrame
skl_theta_dict = {'$\theta_0$': [theta0, ridge_reg.intercept_]}

for i in range(len(ridge_reg.coef_)):
    string = '$\theta_{' + str(i+1) + '}$'
    skl_theta_dict[string] = [theta[i], ridge_reg.coef_[i]]
    
skl_theta_df = pd.DataFrame(skl_theta_dict, index=['Implementacija', 'Scikit-learn']).transpose()
skl_theta_df

U prilog valjanosti modela ide i činjenica da su parametri, priloženi u tabeli iznad, vrlo sličnih vrednosti.\

Primetno je, da parametri $\theta_1$ i $\theta_4$ imaju znatno veće vrednosti od ostalih parametara, što bi značilo da učitan set podataka verovatno većinski potiče od raspodele prediktora $X_1$ i $X_4$ i potencijalno nekih korelisanih odbiraka prediktora $X_2$ i $X_4$, s obzirom na to da parametar $\theta_{13}$ koji odgovara prediktoru $X_2X_4$ ima vrednost preko 11.

